<a href="https://colab.research.google.com/github/ludanzhan/Deep-Learning/blob/main/AlphabetSoupCharity_Optimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


In [4]:
from google.colab import files
uploaded_files = files.upload()

Saving charity_data.csv to charity_data.csv


In [5]:
# read the csv file
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


According to the reseacrh question, the "target" of this model is "IS_SUCCESSFUL" and the features are the rest of the variables

In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df  = application_df.drop(['EIN', 'NAME'], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
countType = application_df['APPLICATION_TYPE'].value_counts()
countType

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace = list(countType[countType < 700].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
countClass = application_df['CLASSIFICATION'].value_counts()
countClass

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
countClass[countClass>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = list(countClass[countClass < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [ ]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df = pd.get_dummies(application_df, 
                          columns=['APPLICATION_TYPE', 'CLASSIFICATION','AFFILIATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'],
                          drop_first=True, dtype=float)

application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns = 'IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
len( X_train_scaled[0])

36

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=7, activation="relu", input_dim=36))

# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=14, input_dim=31,activation="relu"))

# Output layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 259       
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 1)                 15        
                                                                 
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 3s 3ms/step - loss: 0.6140 - accuracy: 0.6861 - val_loss: 0.5845 - val_accuracy: 0.7074
Epoch 2/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5670 - accuracy: 0.7245 - val_loss: 0.5723 - val_accuracy: 0.7126
Epoch 3/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5599 - accuracy: 0.7259 - val_loss: 0.5735 - val_accuracy: 0.7093
Epoch 4/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5571 - accuracy: 0.7273 - val_loss: 0.5690 - val_accuracy: 0.7199
Epoch 5/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7300 - val_loss: 0.5673 - val_accuracy: 0.7155
Epoch 6/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7292 - val_loss: 0.5689 - val_accuracy: 0.7173
Epoch 7/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7308 - val_loss: 0.5674 - val_accuracy: 0.7188

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5597 - accuracy: 0.7234 - 313ms/epoch - 1ms/step
Loss: 0.5596753358840942, Accuracy: 0.7233819365501404


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn2 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=14, activation="sigmoid", input_dim=36))

# Second hidden layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=21, input_dim=31,activation="sigmoid"))

nn2.add(tf.keras.layers.Dense(units=14, input_dim=31,activation="sigmoid"))

# Output layer
#  YOUR CODE GOES HERE

nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 14)                518       
                                                                 
 dense_18 (Dense)            (None, 21)                315       
                                                                 
 dense_19 (Dense)            (None, 14)                308       
                                                                 
 dense_20 (Dense)            (None, 1)                 15        
                                                                 
Total params: 1,156
Trainable params: 1,156
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
#  YOUR CODE GOES HERE
fit_model2 = nn2.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 3s 3ms/step - loss: 0.6430 - accuracy: 0.6414 - val_loss: 0.5914 - val_accuracy: 0.7054
Epoch 2/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5766 - accuracy: 0.7212 - val_loss: 0.5795 - val_accuracy: 0.7160
Epoch 3/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5704 - accuracy: 0.7263 - val_loss: 0.5758 - val_accuracy: 0.7201
Epoch 4/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5672 - accuracy: 0.7261 - val_loss: 0.5734 - val_accuracy: 0.7201
Epoch 5/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5652 - accuracy: 0.7268 - val_loss: 0.5716 - val_accuracy: 0.7204
Epoch 6/100
684/684 [==============================] - 1s 2ms/step - loss: 0.5635 - accuracy: 0.7279 - val_loss: 0.5710 - val_accuracy: 0.7212
Epoch 7/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5613 - accuracy: 0.7287 - val_loss: 0.5697 - val_accuracy: 0.7251

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5534 - accuracy: 0.7254 - 329ms/epoch - 1ms/step
Loss: 0.5533564686775208, Accuracy: 0.7253644466400146


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn3 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn3.add(tf.keras.layers.Dense(units=14, activation="sigmoid", input_dim=36))

# Second hidden layer
#  YOUR CODE GOES HERE
nn3.add(tf.keras.layers.Dense(units=14, input_dim=31,activation="sigmoid"))

nn3.add(tf.keras.layers.Dense(units=14, input_dim=31,activation="sigmoid"))

# Output layer
#  YOUR CODE GOES HERE

nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 14)                518       
                                                                 
 dense_26 (Dense)            (None, 14)                210       
                                                                 
 dense_27 (Dense)            (None, 14)                210       
                                                                 
 dense_28 (Dense)            (None, 1)                 15        
                                                                 
Total params: 953
Trainable params: 953
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
#  YOUR CODE GOES HERE
fit_model3 = nn3.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 2s 3ms/step - loss: 0.6561 - accuracy: 0.6199 - val_loss: 0.5977 - val_accuracy: 0.7046
Epoch 2/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5785 - accuracy: 0.7206 - val_loss: 0.5798 - val_accuracy: 0.7219
Epoch 3/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5700 - accuracy: 0.7266 - val_loss: 0.5745 - val_accuracy: 0.7194
Epoch 4/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5666 - accuracy: 0.7265 - val_loss: 0.5717 - val_accuracy: 0.7256
Epoch 5/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5643 - accuracy: 0.7292 - val_loss: 0.5705 - val_accuracy: 0.7248
Epoch 6/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5623 - accuracy: 0.7304 - val_loss: 0.5686 - val_accuracy: 0.7243
Epoch 7/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5605 - accuracy: 0.7302 - val_loss: 0.5686 - val_accuracy: 0.7243

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5543 - accuracy: 0.7247 - 334ms/epoch - 1ms/step
Loss: 0.5542573928833008, Accuracy: 0.7246647477149963


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
from google.colab import files
nn.save("AlphabetSoupCharity_Optimization.h5")
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>